In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import operator
import gc
import time

import nltk
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
import string
import re

import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix

from keras.layers import add, Dense, Input, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras import backend as K
from keras import models
from keras.engine.topology import Layer
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
N = 500

In [3]:
train = pd.read_csv('./data/train_tweets.csv')
dev = pd.read_csv('./data/dev_tweets.csv')
test = pd.read_csv('./data/test_tweets.csv')

train['tweet'] = train['tweet'].astype(str)
dev['tweet'] = dev['tweet'].astype(str)
test['tweet'] = test['tweet'].astype(str)

In [4]:
print('Train shape:', train.shape)
print('Dev shape  :', dev.shape)
print('Test shape :', test.shape)

Train shape: (96585, 4)
Dev shape  : (34028, 4)
Test shape : (32977, 4)


In [5]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for special in specials:
        text = text.replace(special, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(' ')])
    return text

In [6]:
# remove '@USER_*****'
train['tweet'] = train['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word[0:6] != '@USER_']))
dev['tweet'] = dev['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word[0:6] != '@USER_']))
test['tweet'] = test['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word[0:6] != '@USER_']))

# clean contruction
train['tweet'] = train['tweet'].apply(lambda x: clean_contractions(x, contraction_mapping))
dev['tweet'] = dev['tweet'].apply(lambda x: clean_contractions(x, contraction_mapping))
test['tweet'] = test['tweet'].apply(lambda x: clean_contractions(x, contraction_mapping))

# lower text
train['tweet'] = train['tweet'].apply(lambda x: x.lower())
dev['tweet'] = dev['tweet'].apply(lambda x: x.lower())
test['tweet'] = test['tweet'].apply(lambda x: x.lower())

# remove stop words
train['tweet'] = train['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word not in stopwords]))
dev['tweet'] = dev['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word not in stopwords]))
test['tweet'] = test['tweet'].apply(lambda x: ' '.join([word for word in x.split(' ') if word not in stopwords]))

# remove non alphabetic chars
train['tweet'] = train['tweet'].apply(lambda x: re.sub('[^a-z]+', ' ', x))
dev['tweet'] = dev['tweet'].apply(lambda x: re.sub('[^a-z]+', ' ', x))
test['tweet'] = test['tweet'].apply(lambda x: re.sub('[^a-z]+', ' ', x))

# strip text
train['tweet'] = train['tweet'].apply(lambda x: x.strip())
dev['tweet'] = dev['tweet'].apply(lambda x: x.strip())
test['tweet'] = test['tweet'].apply(lambda x: x.strip())

In [7]:
def merge_same_user(df):
    df_class = df[['user-id', 'class']].groupby('user-id').apply(lambda x: x['class'].mode()).reset_index()
    df_class = df_class.rename(columns={0: 'class'})
    df_new = df[['user-id', 'tweet']].groupby('user-id').agg(lambda x: ' '.join(x))
    df_new = pd.merge(df_new, df_class, on='user-id')
    
    return df_new

In [8]:
train_per_user = merge_same_user(train)
dev_per_user = merge_same_user(dev)
test_per_user = merge_same_user(test)

train_per_user = pd.concat([train_per_user, dev_per_user], axis=0)

In [9]:
print('Train per user shape:', train_per_user.shape)
print('Test per user shape :', test_per_user.shape)

Train per user shape: (3190, 3)
Test per user shape : (802, 3)


In [10]:
word_list = np.load(f'best{N}.npy')

vectorizer = CountVectorizer(vocabulary=word_list)
vectorizer.fit(train_per_user['tweet'])

train_vec = vectorizer.transform(train_per_user['tweet'])
test_vec = vectorizer.transform(test_per_user['tweet'])

train_vec = pd.DataFrame(train_vec.todense(), columns=vectorizer.get_feature_names())
test_vec = pd.DataFrame(test_vec.todense(), columns=vectorizer.get_feature_names())

train_per_user = train_per_user.drop(['tweet'], axis=1)
test_per_user = test_per_user.drop(['tweet'], axis=1)

train_per_user = train_per_user.reset_index(drop=True)
test_per_user = test_per_user.reset_index(drop=True)
train_vec = train_vec.reset_index(drop=True)
test_vec = test_vec.reset_index(drop=True)

train_per_user = pd.concat([train_per_user, train_vec], axis=1)
test_per_user = pd.concat([test_per_user, test_vec], axis=1)

In [11]:
print('Train per user shape:', train_per_user.shape)
print('Test per user shape :', test_per_user.shape)

Train per user shape: (3190, 972)
Test per user shape : (802, 972)


In [12]:
X = train_per_user.drop(['user-id', 'class'], axis=1)
X_test = test_per_user.drop(['user-id', 'class'], axis=1)

# mmsc = MinMaxScaler()
# X = mmsc.fit_transform(X)
# X_test = mmsc.transform(X_test)

ss = StandardScaler()
X = ss.fit_transform(X)
X_test = ss.transform(X_test)

y = train_per_user['class']
encoder = LabelEncoder()
encoder.fit(y)
y_one_hot = encoder.transform(y)
y_one_hot = np_utils.to_categorical(y_one_hot)

In [13]:
folds = StratifiedKFold(n_splits=10, random_state=2019)
fold_num = 1

for train_idx, val_idx in folds.split(X, y):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_one_hot[train_idx], y_one_hot[val_idx]

    gc.collect()
    K.clear_session()

    model = Sequential()
    model.add(Dense(128, input_shape=(X.shape[1],)))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(64))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(32))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=5)
    mcp = ModelCheckpoint(filepath=f'./NN-models/models-bestN/fold-{fold_num}.hdf5', 
                          monitor='val_acc', mode='max', 
                          verbose=1, 
                          save_best_only=True)
    
    model.fit(X_train, y_train,
              validation_data=(X_val, y_val),
              batch_size=32,
              epochs=50,
              verbose=1,
              callbacks=[mcp, es]
             )
    
    fold_num += 1






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2869 samples, validate on 321 samples
Epoch 1/50
2869/2869 [==============================] - 7s 2ms/step - loss: 0.9308 - acc: 0.5674 - val_loss: 0.6319 - val_acc: 0.8006

Epoch 00001: val_acc improved from -inf to 0.80062, saving model to ./models-bestN/fold-1.hdf5
Epoch 2/50
2869/2869 [==============================] - 1s 386us/step - loss: 0.6058 - acc: 0.7633 - val_loss: 0.4799 - val_acc: 0.8318

Epoch 00002: val_acc improved from 0.80062 to 0.83178, saving model to ./models-bestN/fold-1.hdf5
Epoch 3/50
2869/2869 [==============================] - 1s 401us/step - loss: 0.4709 - acc: 0.8167 - val_loss: 0.4372 - val_acc: 0.8442

Epoch 00003: val_acc improved from 0.83178 to 0.84424, saving model to ./models-bestN/fold-1.hdf5
Epoch 4/50
2869/2869 [================

Epoch 2/50
2870/2870 [==============================] - 1s 368us/step - loss: 0.6060 - acc: 0.7606 - val_loss: 0.5051 - val_acc: 0.8156

Epoch 00002: val_acc improved from 0.77812 to 0.81563, saving model to ./models-bestN/fold-4.hdf5
Epoch 3/50
2870/2870 [==============================] - 1s 389us/step - loss: 0.4824 - acc: 0.8160 - val_loss: 0.4505 - val_acc: 0.8406

Epoch 00003: val_acc improved from 0.81563 to 0.84062, saving model to ./models-bestN/fold-4.hdf5
Epoch 4/50
2870/2870 [==============================] - 1s 394us/step - loss: 0.4068 - acc: 0.8422 - val_loss: 0.4282 - val_acc: 0.8438

Epoch 00004: val_acc improved from 0.84062 to 0.84375, saving model to ./models-bestN/fold-4.hdf5
Epoch 5/50
2870/2870 [==============================] - 1s 368us/step - loss: 0.3830 - acc: 0.8551 - val_loss: 0.4247 - val_acc: 0.8562

Epoch 00005: val_acc improved from 0.84375 to 0.85625, saving model to ./models-bestN/fold-4.hdf5
Epoch 6/50
2870/2870 [==============================] - 1s 3

Epoch 2/50
2872/2872 [==============================] - 1s 380us/step - loss: 0.6031 - acc: 0.7535 - val_loss: 0.5903 - val_acc: 0.8302

Epoch 00002: val_acc improved from 0.76415 to 0.83019, saving model to ./models-bestN/fold-8.hdf5
Epoch 3/50
2872/2872 [==============================] - 1s 392us/step - loss: 0.4921 - acc: 0.8022 - val_loss: 0.5531 - val_acc: 0.8302

Epoch 00003: val_acc did not improve from 0.83019
Epoch 4/50
2872/2872 [==============================] - 1s 389us/step - loss: 0.4324 - acc: 0.8329 - val_loss: 0.5508 - val_acc: 0.8176

Epoch 00004: val_acc did not improve from 0.83019
Epoch 5/50
2872/2872 [==============================] - 1s 385us/step - loss: 0.3742 - acc: 0.8590 - val_loss: 0.5540 - val_acc: 0.8208

Epoch 00005: val_acc did not improve from 0.83019
Epoch 6/50
2872/2872 [==============================] - 1s 386us/step - loss: 0.3362 - acc: 0.8653 - val_loss: 0.5532 - val_acc: 0.8270

Epoch 00006: val_acc did not improve from 0.83019
Epoch 7/50
2872/2

In [14]:
val_prob = np.zeros((len(train_per_user), 3), dtype=np.float32)
pred_prob = np.zeros((len(test_per_user), 3), dtype=np.float32)

folds = StratifiedKFold(n_splits=10, random_state=2019)
fold_num = 1

for train_idx, val_idx in folds.split(X, y):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_one_hot[train_idx], y_one_hot[val_idx]

    gc.collect()
    K.clear_session()
    
    model = models.load_model(f'./NN-models/models-bestN/fold-{fold_num}.hdf5')
    val_prob[val_idx] = model.predict(X_val, batch_size=32, verbose=1)
    pred_prob = model.predict(X_test, batch_size=32, verbose=1) / 10
    fold_num += 1

802/802 [==============================] - 0s 51us/step


In [15]:
validation = train_per_user[['user-id', 'class']]
validation['prediction'] = encoder.inverse_transform(np.argmax(val_prob, axis=1))
validation[encoder.classes_[0]] = val_prob[:, 0]
validation[encoder.classes_[1]] = val_prob[:, 1]
validation[encoder.classes_[2]] = val_prob[:, 2]
validation.to_csv(f'./result-per-user/cnn-best{N}-val.csv', index=False)
accuracy_score(validation['class'], validation['prediction'])

0.8385579937304075

In [16]:
submission = test_per_user[['user-id', 'class']]
submission['prediction'] = encoder.inverse_transform(np.argmax(pred_prob, axis=1))
submission[encoder.classes_[0]] = pred_prob[:, 0]
submission[encoder.classes_[1]] = pred_prob[:, 1]
submission[encoder.classes_[2]] = pred_prob[:, 2]
submission.to_csv(f'./result-per-user/cnn-best{N}-sub.csv', index=False)